# 数据聚合

对于聚合，指的是任何能够从数组产生标量值的数据转换过程。

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})

df

,data1,data2,key1,key2
0,-0.794780,-0.054811,a,one
1,-1.962545,0.244446,a,two
2,-0.886427,-0.367513,b,one
3,-0.397833,3.000682,b,two
4,0.494439,0.398436,a,one


In [3]:
grouped = df.groupby('key1')

grouped

In [4]:
grouped['data1'].quantile(0.9)

key1
a    0.236595
b   -0.446692
Name: data1, dtype: float64

In [5]:
type(grouped['data1'].quantile(0.9))

pandas.core.series.Series

如果要使用你自己的聚合函数，只需要将其传入aggregate或agg方法即可：

In [6]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [7]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.456984,0.453246
b,0.488594,3.368196


In [8]:
grouped.describe()

data1                                                             \
     count      mean       std       min       25%      50%       75%   
key1                                                                    
a      3.0 -0.754296  1.228992 -1.962545 -1.378663 -0.79478 -0.150171   
b      2.0 -0.642130  0.345488 -0.886427 -0.764278 -0.64213 -0.519981   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.494439   3.0  0.196024  0.230470 -0.054811  0.094817  0.244446   
b    -0.397833   2.0  1.316585  2.381674 -0.367513  0.474536  1.316585   

                          
           75%       max  
key1                      
a     0.321441  0.398436  
b     2.158633  3.000682

In [9]:
tips = pd.read_csv('./Data/tips.csv')

tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [10]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


## 面向列的多函数应用

In [11]:
grouped = tips.groupby(['sex', 'smoker'])

grouped_pct = grouped['tip_pct']

grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [12]:
tips['tip_pct'].groupby([tips['sex'], tips['smoker']]).mean()

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

如果传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名：

In [13]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

如果传入的是一个由（name，function）元祖组成的列表，则各元组的第一个元素就会被用作DataFrame的列名（可以将这种二元元组列表看做一个有序映射）：

In [14]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

对于DataFrame，你还可以定义一组应用于全部列的函数，或不同的列应用不同的函数。假设我们想要对tip_pct和total_bill列计算三个统计信息：

In [15]:
functions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(functions)

result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [16]:
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

In [17]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

假设你想要对不同的列应用不同的函数。具体的办法是向agg传入一个从列名映射到函数的字典：

In [18]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [19]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

In [20]:
grouped.agg({'tip_pct': [('my_min', 'min'),'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                 my_min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

## 以“无索引”的形式返回聚合数据

In [21]:
tips.groupby(['sex', 'smoker']).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

In [22]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771
